In [1]:
import os 
import openai
import pandas as pd

In [5]:
openai.api_key = os.environ["OPENAI_KEY"]

In [2]:
df = pd.read_csv("./data/sales_data_sample.csv")

In [3]:
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,Shipped,4,12,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,Shipped,1,1,2005,...,Torikatu 38,NaN,Oulu,NaN,90110,Finland,EMEA,Koskitalo,Pirkko,Medium
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,Resolved,1,3,2005,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,Shipped,1,3,2005,...,1 rue Alsace-Lorraine,NaN,Toulouse,NaN,31000,France,EMEA,Roulet,Annette,Small


In [9]:
# What was the toatl sum of sales per quarter?
# -> select sum(sales) where quarter="4";

In [11]:
from sqlalchemy import create_engine

In [12]:
from sqlalchemy import text

In [13]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [14]:
data = df.to_sql(name="sales", con=temp_db)

2023-08-12 23:14:26,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-12 23:14:26,025 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2023-08-12 23:14:26,025 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 23:14:26,026 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sales")
2023-08-12 23:14:26,027 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-12 23:14:26,028 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-08-12 23:14:26,029 INFO sqlalchemy.engine.Engine [no key 0.00069s] ()
2023-08-12 23:14:26,030 INFO sqlalchemy.engine.Engine CREATE INDEX ix_sales_index ON sales ("index

In [21]:
with temp_db.connect() as conn:
    result = conn.execute(text("select sum(SALES) from sales limit 100"))
    

2023-08-12 23:18:54,806 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-12 23:18:54,807 INFO sqlalchemy.engine.Engine select sum(SALES) from sales limit 100
2023-08-12 23:18:54,808 INFO sqlalchemy.engine.Engine [generated in 0.00158s] ()
2023-08-12 23:18:54,809 INFO sqlalchemy.engine.Engine ROLLBACK


In [22]:
result.all()

[(10032628.85000001,)]

In [32]:
def create_table_def(df):
    promp = f"""### sqlite SQL table, with its properites:
    #
    # sales({ ','.join( str(col) for col in df.columns )})
    # """ 
    return promp


In [33]:
print(create_table_def(df))

### sqlite SQL table, with its properites:
    #
    # sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    # 


In [35]:
def prompt_input():
    nlp = input(">> enter the query")
    return nlp

In [37]:
def combine_prompts(df, query):
    definition = create_table_def(df)
    query_str = f"### A query to answer: {query}\n SELECT"
    return f"{definition}{query_str}"

In [49]:
def patch_response_query(res, opt="SELECT"):
    query = res['choices'][0]['text']
    query = f"{opt} {query.strip()}"
    return query

In [56]:
def query_db(sql):
    with temp_db.connect() as conn:
        result = conn.execute(text(sql))
        return result

In [51]:
nlp = prompt_input()
combine_prompts(df, nlp)

>> enter the query return the sum of sales by postalcode


'### sqlite SQL table, with its properites:\n    #\n    # sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    # ### A query to answer: return the sum of sales by postalcode\n SELECT'

In [52]:
res = openai.Completion.create(model="text-davinci-003",
                               prompt = combine_prompts(df, nlp),
                               temperature = 0,
                               max_tokens = 150,
                               top_p = 1.0,
                               frequency_penalty=0,
                               presence_penalty=0,
                               stop = ["#", ";"])

In [53]:
res

<OpenAIObject text_completion id=cmpl-7mzE5xwN2zw0Z0Y1crwREcGSd6D1O at 0x21b347fe450> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-7mzE5xwN2zw0Z0Y1crwREcGSd6D1O",
  "object": "text_completion",
  "created": 1691909453,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": " POSTALCODE, SUM(SALES)\n FROM sales\n GROUP BY POSTALCODE",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 119,
    "completion_tokens": 21,
    "total_tokens": 140
  }
}

In [54]:
patch_response_query(res)

'SELECT POSTALCODE, SUM(SALES)\n FROM sales\n GROUP BY POSTALCODE'

In [58]:
data = query_db(patch_response_query(res))

2023-08-12 23:53:01,825 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-12 23:53:01,826 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES)
 FROM sales
 GROUP BY POSTALCODE
2023-08-12 23:53:01,826 INFO sqlalchemy.engine.Engine [cached since 17.4s ago] ()
2023-08-12 23:53:01,828 INFO sqlalchemy.engine.Engine ROLLBACK


In [59]:
data.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287